\pagebreak


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pandas import DataFrame as df
from statannotations.Annotator import Annotator

import latstability as ls

def step_width_batch(inputdf, event_channels, y_channels):
    """Doing step width calculation in one go
    :param inputdf: A spike file input as *.csv or formatted as such
    :param event_channels: A list with all the proper channel names for the event channels
    :param y_channels: A list with all the proper channel names for the channels from DLC
    :note: The proper order for event channels goes lhl, lfl, rhl, rfl with swonset first.

    :return fl_step_widths: array of step width values for the forelimb
    :return hl_step_widths: array of step width values for the hindlimb
    """

    lhl_st_lengths, lhl_st_timings = ls.stance_duration(
        inputdf, swonset_channel=event_channels[0], swoffset_channel=event_channels[1]
    )
    lfl_st_lengths, lfl_st_timings = ls.stance_duration(
        inputdf, swonset_channel=event_channels[2], swoffset_channel=event_channels[3]
    )
    rhl_st_lengths, rhl_st_timings = ls.stance_duration(
        inputdf, swonset_channel=event_channels[4], swoffset_channel=event_channels[5]
    )
    rfl_st_lengths, rfl_st_timings = ls.stance_duration(
        inputdf, swonset_channel=event_channels[6], swoffset_channel=event_channels[7]
    )

    # For forelimb
    fl_step_widths = ls.step_width(
        inputdf, rfl_st_timings, lfl_st_timings, rl_y=y_channels[0], ll_y=y_channels[1]
    )
    hl_step_widths = ls.step_width(
        inputdf, rhl_st_timings, lhl_st_timings, rl_y=y_channels[2], ll_y=y_channels[3]
    )

    return fl_step_widths, hl_step_widths

def condition_add(input_df, file_list, condition, limb, perturbation_state):
    for i in range(len(file_list)):
        limb = limb
        perturbation_state = perturbation_state
        mos_values = pd.read_csv(file_list[i], header=None)
        mos_values = mos_values.to_numpy()
        mos_values = mos_values.ravel()
        for j in range(len(mos_values)):
            entry = mos_values[j]
            mos_entry = [[condition, limb, perturbation_state, entry]]

            input_df = input_df._append(
                pd.DataFrame(
                    mos_entry,
                    columns=["Condition", "Limb", "Perturbation State", "MoS"],
                ),
                ignore_index=True,
            )

    return input_df

# Introduction

For most terrestrial mammals, locomotion is essential for basic survival and gene propagation. Therefore, understanding the neural mechanisms that govern locomotion has been a field of interest with many advancements over the years @Frigon2021. In the central nervous system (CNS) there are two main regions; the brain and spinal cord. 


# Methods
<!--- Check to see if we included mice that never make in into data --->

Our study included 18 adult mice C57BL/6 (aged ) 

## EMG Implantation Surgeries

While any data pertaining to EMG activity is not presented in this thesis, it should be noted all mice underwent EMG implantation surgeries. This included the implantation of eight bipolar electrodes [@Pearson2005] in the following muscles; Gluteus Maximus (GM), Iliopsoas (Ip), Biceps Femoris (BF), Vastus Lateralis (VL), Semitendinosus (St), Tibialis Anterior (TA), Gastrocnemius (Gs), Gracilis (Gr). Before the implantation mice were given analgesics Buprenorphine Slow Release (SR) and Meloxicam at 0.1 mg/kg and were anaesthetized with isofluorane at a concentration between 2-3% throughout the procedure. 

## Kinematic Recordings

Kinematic recordings were taken on a high-speed camera at a resolution of 1280 $\times$ 800 at 500 fps (IL3; Fastec Imaging, San Diego, CA, USA). To track limb movement as well as CoM position we used the markerless tracking software DeepLabCut, v2.9.0 [@Mathis2018]. To record both a sagittal and horizontal  view of mouse we used a custom treadmill with transparent tread and mirror showing the belly.

## Diphtheria Toxin (DTX) Administration

For the 8 *PV^Cre^::Avil^iDTR^::Rosa^EGFP^* mice we performed intraperitoneal injection of diphtheria injection of DTX (D0654 lyophilized; Sigma-Aldrich, St Louis, MO, USA) after performing the first set of EMG & kinematic recordings. We injected the toxin at a concentration of 100 $\mu$g kg^-1^ diluted in pure water as previously described in [@Takeoka2019]. Mice were quarantined for 3 days before performing the second set of kinematic and EMG recordings on day 7 post injection.

# Results

## Fig 1: Hip Height and Step width are determinate of stability and regulated by proprioceptive feedback

- This will show how to the ground the CoM is for each condition. Lower hip height in proprioceptive loss may indicate the use of lowering center of gravity as a compensatory mechanism to offset the lack of feedback?
- Looking at step width comparison could give some insight to the large scale of variability seen in the post-DTX mice. This could suggest something regarding the role of GTO's in signaling limb position for the purpose of coordination. 


In [ ]:
# Wild type data
wt_event_channels = {
    "wt_1": [
        "51 HLl Sw on",
        "52 HLl Sw of",
        "55 FLl Sw on",
        "56 FLl Sw of",
        "53 HLr Sw on",
        "54 HLr Sw of",
        "57 FLr Sw on",
        "58 FLr Sw of",
    ],
    "wt_2": [
        "51 lHLswon",
        "52 lHLswoff",
        "55 lFLswon",
        "56 lFLswoff",
        "53 rHLswon",
        "54 rHLswoff",
        "57 rFLswon",
        "58 rFLswoff",
    ],
    "wt_3": [
        "56 lHL swon",
        "57 lHL swoff",
        "52 lFL swon",
        "53 lFL swoff",
        "58 rHL swon",
        "59 rHL swoff",
        "54 rFL swon",
        "55 rFL swoff",
    ],
    "wt_4": [
        "57 lHL swon",
        "58 lHL swoff",
        "53 lFL swon",
        "54 lFL swoff",
        "55 rHL swon",
        "56 rHL swoff",
        "51 rFL swon",
        "52 rFL swoff",
    ],
    "wt_5": [
        "48 lHL swon",
        "49 lHL swoff",
        "44 lFL swon",
        "45 lFL swoff",
        "46 rHL swon",
        "47 rHL swoff",
        "42 rFL swon",
        "43 rFL swoff",
    ],
}

wt_raw = {
    "wt_1": [
        "./wt_data/wt-1-non-all.txt",
        "./wt_data/wt-1-per-all.txt",
        "./wt_data/wt-1-sin-all.txt",
    ],
    "wt_2": [
        "./wt_data/wt-2-non-all.txt",
        "./wt_data/wt-2-per-all.txt",
        "./wt_data/wt-2-sin-all.txt",
    ],
    "wt_3": [
        "./wt_data/wt-3-non-all.txt",
        "./wt_data/wt-3-per-all.txt",
        "./wt_data/wt-3-sin-all.txt",
    ],
    "wt_4": [
        "./wt_data/wt-4-non-all.txt",
        "./wt_data/wt-4-per-all.txt",
        "./wt_data/wt-4-sin-all.txt",
    ],
}


wt_y_channels = ["35 FRy (cm)", "33 FLy (cm)", "30 HRy (cm)", "28 HLy (cm)"]

wt5nondf = pd.read_csv("./wt_data/wt-5-non-all.txt", delimiter=",", header=0)
wt5perdf = pd.read_csv("./wt_data/wt-5-per-all.txt", delimiter=",", header=0)


# Step Width Calculation
conditions = [
    "Non-Perturbation",
    "Perturbation",
    "Sinusoidal",
]

for i in wt_raw:
    print(f"mouse being considered is {i}")
    mouse_data = wt_raw[i]
    # print(wt_event_channels[i])
    for j in range(len(mouse_data)):
        event_channels = wt_event_channels[i]  # getting channel names
        current_condtion = conditions[j]  # simply to keep track
        data_path = mouse_data[j]  # file from spike for condition
        trial_spike = pd.read_csv(data_path, delimiter=",", header=0)
        fl_stepw, hl_stepw = step_width_batch(
            trial_spike, event_channels=event_channels, y_channels=wt_y_channels
        )
        print(f"forelimb step width for {current_condtion} is {fl_stepw}")
        print(f"hindlimb step width for {current_condtion} is {hl_stepw}")

    print()

# M5 Does not have a sinusoidal recording so
# Non-Perturbation
wt_5_non_fl, wt_5_non_hl = step_width_batch(
    wt5nondf, wt_event_channels["wt_5"], wt_y_channels
)
print(wt_5_non_fl)
print(wt_5_non_hl)

# Perturbation
wt_5_per_fl, wt_5_per_hl = step_width_batch(
    wt5perdf, wt_event_channels["wt_5"], wt_y_channels
)

print(wt_5_per_fl)
print(wt_5_per_hl)

# Egr3 Mice
egr3_event_channels = {
    "egr3_6": [
        "51 HLl Sw on",
        "52 HLl Sw of",
        "55 FLl Sw on",
        "56 FLl Sw of",
        "53 HLr Sw on",
        "54 HLr Sw of",
        "57 FLr Sw on",
        "58 FLr Sw of",
    ],
    "egr3_7": [
        "51 lHLswon",
        "52 lHLswoff",
        "55 lFLswon",
        "56 lFLswoff",
        "53 rHLswon",
        "54 rHLswoff",
        "57 rFLswon",
        "58 rFLswoff",
    ],
    "egr3_8": [
        "56 lHL swon",
        "57 lHL swoff",
        "52 lFL swon",
        "53 lFL swoff",
        "58 rHL swon",
        "59 rHL swoff",
        "54 rFL swon",
        "55 rFL swoff",
    ],
    "egr3_9": [
        "57 lHL swon",
        "58 lHL swoff",
        "53 lFL swon",
        "54 lFL swoff",
        "55 rHL swon",
        "56 rHL swoff",
        "51 rFL swon",
        "52 rFL swoff",
    ],
    "egr3_10": [
        "48 lHL swon",
        "49 lHL swoff",
        "44 lFL swon",
        "45 lFL swoff",
        "46 rHL swon",
        "47 rHL swoff",
        "42 rFL swon",
        "43 rFL swoff",
    ],
}

egr3_raw = {
    "egr3_6": [
        "./egr3_data/egr3-6-non-all.txt",
        "./egr3_data/egr3-6-per-all.txt",
        "./egr3_data/egr3-6-sin-all.txt",
    ],
    "egr3_7": [
        "./egr3_data/egr3-7-non-all.txt",
        "./egr3_data/egr3-7-per-all.txt",
        "./egr3_data/egr3-7-sin-all.txt",
    ],
    "egr3_8": [
        "./egr3_data/egr3-8-non-all.txt",
        "./egr3_data/egr3-8-per-all.txt",
        "./egr3_data/egr3-8-sin-all.txt",
    ],
    "egr3_9": [
        "./egr3_data/egr3-9-non-all.txt",
        "./egr3_data/egr3-9-per-all-2.txt",
        "./egr3_data/egr3-9-sin-all-1.txt",
    ],
}


egr3_y_channels = ["35 FRy (cm)", "33 FLy (cm)", "30 HRy (cm)", "28 HLy (cm)"]

# for i in egr3_raw:
#     print(f"mouse being considered is {i}")
#     mouse_data = egr3_raw[i]
#     # print(wt_event_channels[i])
#     for j in range(len(mouse_data)):
#         event_channels = egr3_event_channels[i]  # getting channel names
#         current_condtion = conditions[j]  # simply to keep track
#         data_path = mouse_data[j]  # file from spike for condition
#         trial_spike = pd.read_csv(data_path, delimiter=",", header=0)
#         fl_stepw, hl_stepw = step_width_batch(
#             trial_spike, event_channels=event_channels, y_channels=wt_y_channels
#         )
#         print(f"forelimb step width for {current_condtion} is {fl_stepw}")
#         print(f"hindlimb step width for {current_condtion} is {hl_stepw}")
#
#     print()

## Postural stability in the Wild-type mouse

### Step width and hip height in Wild-type mouse

## Role of Muscle Spindles in Postural Stability

## Role of Golgi Tendon Organs in Postural Stability


In [ ]:
wt_non_lmos = [
    "./wt_data/wt1non_lmos.csv",
    "./wt_data/wt2non_lmos.csv",
    "./wt_data/wt3non_lmos.csv",
    "./wt_data/wt4non_lmos.csv",
    "./wt_data/wt5non_lmos.csv",
]

wt_non_rmos = [
    "./wt_data/wt1non_rmos.csv",
    "./wt_data/wt2non_rmos.csv",
    "./wt_data/wt3non_rmos.csv",
    "./wt_data/wt4non_rmos.csv",
    "./wt_data/wt5non_rmos.csv",
]

wt_per_lmos = [
    "./wt_data/wt1per_lmos.csv",
    "./wt_data/wt2per_lmos.csv",
    "./wt_data/wt3per_lmos.csv",
    "./wt_data/wt4per_lmos.csv",
    "./wt_data/wt5per_lmos.csv",
]

wt_per_rmos = [
    "./wt_data/wt1per_rmos.csv",
    "./wt_data/wt2non_rmos.csv",
    "./wt_data/wt3per_rmos.csv",
    "./wt_data/wt4per_rmos.csv",
    "./wt_data/wt5per_rmos.csv",
]

wt_sin_lmos = [
    "./wt_data/wt1sin_lmos.csv",
    "./wt_data/wt2sin_lmos.csv",
    "./wt_data/wt3sin_lmos.csv",
    "./wt_data/wt4sin_lmos.csv",
]

wt_sin_rmos = [
    "./wt_data/wt1sin_rmos.csv",
    "./wt_data/wt2sin_rmos.csv",
    "./wt_data/wt3sin_rmos.csv",
    "./wt_data/wt4sin_rmos.csv",
]

# For Egr3
egr3_non_lmos = [
    "./egr3_data/egr3_6non_lmos.csv",
    "./egr3_data/egr3_7non_lmos.csv",
    "./egr3_data/egr3_8non_lmos.csv",
    "./egr3_data/egr3_9non_lmos.csv",
    "./egr3_data/egr3_10non_lmos.csv",
]

egr3_non_rmos = [
    "./egr3_data/egr3_6non_rmos.csv",
    "./egr3_data/egr3_7non_rmos.csv",
    "./egr3_data/egr3_8non_rmos.csv",
    "./egr3_data/egr3_9non_rmos.csv",
    "./egr3_data/egr3_10non_rmos.csv",
]

egr3_per_lmos = [
    "./egr3_data/egr3_6per_lmos.csv",
    "./egr3_data/egr3_7per_lmos.csv",
    "./egr3_data/egr3_8per_lmos.csv",
    "./egr3_data/egr3_9per_lmos-1.csv",
    "./egr3_data/egr3_9per_lmos-2.csv",
    "./egr3_data/egr3_10per_lmos-1.csv",
    "./egr3_data/egr3_10per_lmos-2.csv",
]

egr3_per_rmos = [
    "./egr3_data/egr3_6per_rmos.csv",
    "./egr3_data/egr3_7per_rmos.csv",
    "./egr3_data/egr3_8per_rmos.csv",
    "./egr3_data/egr3_9per_rmos-1.csv",
    "./egr3_data/egr3_9per_rmos-2.csv",
    "./egr3_data/egr3_10per_rmos-1.csv",
    "./egr3_data/egr3_10per_rmos-2.csv",
]

egr3_sin_lmos = [
    "./egr3_data/egr3_6sin_lmos.csv",
    "./egr3_data/egr3_7sin_lmos.csv",
    "./egr3_data/egr3_8sin_lmos.csv",
    "./egr3_data/egr3_9sin_lmos-1.csv",
    "./egr3_data/egr3_9sin_lmos-2.csv",
    "./egr3_data/egr3_10sin_lmos.csv",
]

egr3_sin_rmos = [
    "./egr3_data/egr3_6sin_rmos.csv",
    "./egr3_data/egr3_7sin_rmos.csv",
    "./egr3_data/egr3_8sin_rmos.csv",
    "./egr3_data/egr3_9sin_rmos-1.csv",
    "./egr3_data/egr3_9sin_rmos-2.csv",
    "./egr3_data/egr3_10sin_rmos.csv",
]


dtrpre_non_lmos = [
    "./dtr_data/predtx/predtx_2non_lmos.csv",
    "./dtr_data/predtx/predtx_3non_lmos.csv",
    "./dtr_data/predtx/predtx_5non_lmos.csv",
    "./dtr_data/predtx/predtx_6non_lmos.csv",
    "./dtr_data/predtx/predtx_7non_lmos.csv",
]

dtrpre_non_rmos = [
    "./dtr_data/predtx/predtx_2non_rmos.csv",
    "./dtr_data/predtx/predtx_3non_rmos.csv",
    "./dtr_data/predtx/predtx_5non_rmos.csv",
    "./dtr_data/predtx/predtx_6non_rmos.csv",
    "./dtr_data/predtx/predtx_7non_rmos.csv",
]


dtrpre_per_lmos = [
    "./dtr_data/predtx/predtx_2per_lmos.csv",
    "./dtr_data/predtx/predtx_3per_lmos.csv",
    "./dtr_data/predtx/predtx_5per_lmos-1.csv",
    "./dtr_data/predtx/predtx_5per_lmos-2.csv",
    "./dtr_data/predtx/predtx_6per_lmos.csv",
    "./dtr_data/predtx/predtx_7per_lmos.csv",
]

dtrpre_per_rmos = [
    "./dtr_data/predtx/predtx_2per_rmos.csv",
    "./dtr_data/predtx/predtx_3per_rmos.csv",
    "./dtr_data/predtx/predtx_5per_rmos-1.csv",
    "./dtr_data/predtx/predtx_5per_rmos-2.csv",
    "./dtr_data/predtx/predtx_6per_rmos.csv",
    "./dtr_data/predtx/predtx_7per_rmos.csv",
]

dtrpre_sin_lmos = [
    "./dtr_data/predtx/predtx_2sin_lmos.csv",
    "./dtr_data/predtx/predtx_3sin_lmos-1.csv",
    "./dtr_data/predtx/predtx_3sin_lmos-2.csv",
    "./dtr_data/predtx/predtx_5sin_lmos.csv",
    "./dtr_data/predtx/predtx_6sin_lmos.csv",
    "./dtr_data/predtx/predtx_7sin_lmos.csv",
]

dtrpre_sin_rmos = [
    "./dtr_data/predtx/predtx_2sin_rmos.csv",
    "./dtr_data/predtx/predtx_3sin_rmos-1.csv",
    "./dtr_data/predtx/predtx_3sin_rmos-2.csv",
    "./dtr_data/predtx/predtx_5sin_rmos.csv",
    "./dtr_data/predtx/predtx_6sin_rmos.csv",
    "./dtr_data/predtx/predtx_7sin_rmos.csv",
]

dtrpost_non_lmos = [
    "./dtr_data/postdtx/postdtx_2non_lmos.csv",
    "./dtr_data/postdtx/postdtx_3non_lmos.csv",
    "./dtr_data/postdtx/postdtx_5non_lmos.csv",
    "./dtr_data/postdtx/postdtx_6non_lmos.csv",
]

dtrpost_non_rmos = [
    "./dtr_data/postdtx/postdtx_2non_rmos.csv",
    "./dtr_data/postdtx/postdtx_3non_rmos.csv",
    "./dtr_data/postdtx/postdtx_5non_rmos.csv",
    "./dtr_data/postdtx/postdtx_6non_rmos.csv",
]

dtrpost_per_lmos = [
    "./dtr_data/postdtx/postdtx_2per_lmos.csv",
    "./dtr_data/postdtx/postdtx_3per_lmos.csv",
    "./dtr_data/postdtx/postdtx_5per_lmos-1.csv",
    "./dtr_data/postdtx/postdtx_5per_lmos-2.csv",
    "./dtr_data/postdtx/postdtx_6per_lmos-auto.csv",
]

dtrpost_per_rmos = [
    "./dtr_data/postdtx/postdtx_2per_rmos.csv",
    "./dtr_data/postdtx/postdtx_3per_rmos.csv",
    "./dtr_data/postdtx/postdtx_5per_rmos-1.csv",
    "./dtr_data/postdtx/postdtx_5per_rmos-2.csv",
    "./dtr_data/postdtx/postdtx_6per_rmos-auto.csv",
]

dtrpost_sin_lmos = [
    "./dtr_data/postdtx/postdtx_2sin_lmos.csv",
    # "./dtr_data/postdtx/postdtx_3sin_lmos.csv",
    "./dtr_data/postdtx/postdtx_5sin_lmos.csv",
    "./dtr_data/postdtx/postdtx_6sin_lmos-man.csv",
]

dtrpost_sin_rmos = [
    "./dtr_data/postdtx/postdtx_2sin_rmos.csv",
    # "./dtr_data/postdtx/postdtx_3sin_rmos.csv",
    "./dtr_data/postdtx/postdtx_5sin_rmos.csv",
    "./dtr_data/postdtx/postdtx_6sin_rmos-man.csv",
]

mos_df = df(columns=["Condition", "Limb", "Perturbation State", "MoS"])

mos_df = condition_add(mos_df, wt_non_lmos, "WT", "Left", "Non-Perturbation")
mos_df = condition_add(mos_df, wt_non_rmos, "WT", "Right", "Non-Perturbation")
mos_df = condition_add(mos_df, wt_per_lmos, "WT", "Left", "Perturbation")
mos_df = condition_add(mos_df, wt_per_rmos, "WT", "Right", "Perturbation")
mos_df = condition_add(mos_df, wt_sin_lmos, "WT", "Left", "Sinusoidal")
mos_df = condition_add(mos_df, wt_sin_rmos, "WT", "Right", "Sinusoidal")

mos_df = condition_add(mos_df, egr3_non_lmos, "Egr3", "Left", "Non-Perturbation")
mos_df = condition_add(mos_df, egr3_non_rmos, "Egr3", "Right", "Non-Perturbation")
mos_df = condition_add(mos_df, egr3_per_lmos, "Egr3", "Left", "Perturbation")
mos_df = condition_add(mos_df, egr3_per_rmos, "Egr3", "Right", "Perturbation")
mos_df = condition_add(mos_df, egr3_sin_lmos, "Egr3", "Left", "Sinusoidal")
mos_df = condition_add(mos_df, egr3_sin_rmos, "Egr3", "Right", "Sinusoidal")

mos_df = condition_add(mos_df, dtrpre_non_lmos, "Pre-DTX", "Left", "Non-Perturbation")
mos_df = condition_add(mos_df, dtrpre_non_rmos, "Pre-DTX", "Right", "Non-Perturbation")
mos_df = condition_add(mos_df, dtrpre_per_lmos, "Pre-DTX", "Left", "Perturbation")
mos_df = condition_add(mos_df, dtrpre_per_rmos, "Pre-DTX", "Right", "Perturbation")
mos_df = condition_add(mos_df, dtrpre_sin_lmos, "Pre-DTX", "Left", "Sinusoidal")
mos_df = condition_add(mos_df, dtrpre_sin_rmos, "Pre-DTX", "Right", "Sinusoidal")

mos_df = condition_add(mos_df, dtrpost_non_lmos, "Post-DTX", "Left", "Non-Perturbation")
mos_df = condition_add(
    mos_df, dtrpost_non_rmos, "Post-DTX", "Right", "Non-Perturbation"
)
mos_df = condition_add(mos_df, dtrpost_per_lmos, "Post-DTX", "Left", "Perturbation")
mos_df = condition_add(mos_df, dtrpost_per_rmos, "Post-DTX", "Right", "Perturbation")
mos_df = condition_add(mos_df, dtrpost_sin_lmos, "Post-DTX", "Left", "Sinusoidal")
mos_df = condition_add(mos_df, dtrpost_sin_rmos, "Post-DTX", "Right", "Sinusoidal")

# For just comparing between perturbation
mos_combo = mos_df.drop(columns=["Limb"])
con_mos_combo = mos_df.drop(columns=["Limb"])

con_mos_combo.to_csv("./mos_limbs_combined_all.csv")

# Plotting
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set(style="white", font_scale=1.5, rc=custom_params)

# Dual plot figure was nice until 4 conditions happened
# fig, axs = plt.subplots(1, 2)

# limb_pairs = [
#     [("Left", "Non-Perturbation"), ("Left", "Perturbation")],
#     [("Left", "Non-Perturbation"), ("Left", "Sinusoidal")],
#     [("Right", "Non-Perturbation"), ("Right", "Perturbation")],
#     [("Right", "Non-Perturbation"), ("Right", "Sinusoidal")],
# ]

combo_pairs = [
    [("Non-Perturbation"), ("Perturbation")],
]
combo_legend = ["Non-Perturbation", "Perturbation", "Sinusoidal"]


perturbation_state_order = ["Non-Perturbation", "Perturbation", "Sinusoidal"]

# # Plot for limb comparison
# limb_plot_params = {
#     "data": mos_df,
#     "x": "Limb",
#     "y": "MoS",
#     "hue": "Perturbation State",
#     "hue_order": perturbation_state_order,
# }
#
plt.title("MoS between WT, Egr3 KO, and DTX Mice Pre and Post Injection")
#
# axs[0].set_title("MoS for Egr3 by Limb")
# limb_comp = sns.violinplot(**limb_plot_params, ci=95, capsize=0.05, ax=axs[0])
# # axs[0].legend(fontsize=12, bbox_to_anchor=(2.49, 0.7))
# axs[0].legend(fontsize=12, loc="best")
# annotator = Annotator(limb_comp, limb_pairs, **limb_plot_params)
# annotator.new_plot(limb_comp, limb_pairs, plot="violinplot", **limb_plot_params)
# annotator.configure(
#     hide_non_significant=False, test="t-test_ind", text_format="star", loc="inside"
# )
# annotator.apply_test().annotate(line_offset_to_group=0.2, line_offset=0.1)

# Intercondition Comparison
condition_pairs = [
    # Comparison within wildtype condition
    [("WT", "Non-Perturbation"), ("WT", "Perturbation")],
    [("WT", "Sinusoidal"), ("WT", "Perturbation")],
    [("WT", "Non-Perturbation"), ("WT", "Sinusoidal")],
    # Comparison between Wildtype and Pre-DTX
    [("WT", "Non-Perturbation"), ("Pre-DTX", "Non-Perturbation")],
    [("WT", "Sinusoidal"), ("Pre-DTX", "Sinusoidal")],
    [("WT", "Perturbation"), ("Pre-DTX", "Perturbation")],
    # Comparison within Egr3 condition
    [("Egr3", "Non-Perturbation"), ("Egr3", "Perturbation")],
    [("Egr3", "Sinusoidal"), ("Egr3", "Perturbation")],
    [("Egr3", "Non-Perturbation"), ("Egr3", "Sinusoidal")],
    # Comparison within Pre-DTX condition
    [("Pre-DTX", "Non-Perturbation"), ("Pre-DTX", "Perturbation")],
    [("Pre-DTX", "Sinusoidal"), ("Pre-DTX", "Perturbation")],
    [("Pre-DTX", "Non-Perturbation"), ("Pre-DTX", "Sinusoidal")],
    # Comparison within Post-DTX condition
    [("Post-DTX", "Non-Perturbation"), ("Post-DTX", "Perturbation")],
    [("Post-DTX", "Sinusoidal"), ("Post-DTX", "Perturbation")],
    [("Post-DTX", "Non-Perturbation"), ("Post-DTX", "Sinusoidal")],
]

perturbation_state_order = ["Non-Perturbation", "Perturbation", "Sinusoidal"]
cond_combo_plot_params = {
    "data": con_mos_combo,
    "x": "Condition",
    "y": "MoS",
    "hue": "Perturbation State",
    "hue_order": perturbation_state_order,
    "inner": "point",
}

# axs[0].set_title("MoS between conditions")
cond_combo_comp = sns.violinplot(**cond_combo_plot_params, ci=95, capsize=0.05)
plt.axhline(y=0.0, color='r', linestyle='-')
plt.legend(loc="best", fontsize=12)
annotator = Annotator(cond_combo_comp, condition_pairs, **cond_combo_plot_params)
annotator.new_plot(
    cond_combo_comp, condition_pairs, plot="violinplot", **cond_combo_plot_params
)
annotator.configure(
    hide_non_significant=True, test="t-test_ind", text_format="star", loc="inside"
)

annotator.apply_test().annotate(line_offset_to_group=0.2, line_offset=0.1)
# axs[1].set_title("MoS between conditions")
# cond_combo_comp = sns.violinplot(
#     **cond_combo_plot_params, ci=95, capsize=0.05, ax=axs[1]
# )
# axs[1].legend(loc="upper right", fontsize=12)
# annotator = Annotator(cond_combo_comp, condition_pairs, **cond_combo_plot_params)
# annotator.new_plot(
#     cond_combo_comp, condition_pairs, plot="violinplot", **cond_combo_plot_params
# )
# annotator.configure(
#     hide_non_significant=True, test="t-test_ind", text_format="star", loc="inside"
# )
# annotator.apply_test().annotate(line_offset_to_group=0.2, line_offset=0.1)

# plt.show()

## Proprioception is Necessary for Maintenance of Postural Stability

While lower MoS implies a less stable gait, 

# Discussion

<!--- 
- So what? 
- Did we address the 
--->

We showed interestingly that in the case of selective muscle spindles ablation, we see an interesting trend where MoS is not changing regardless of perturbation being applied indicating a general perturbed state. This corroborates previous findings in EMG from [@Santuz2022b]


\pagebreak{}

# References

:::{#refs}
:::